In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV

from mlxtend.feature_selection import SequentialFeatureSelector

import pickle





import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')

In [ ]:
SEED = 42

# Data

For our modeling approaches, certain models would require lagged data while other, time-series oriented models, would be able to use unlagged data.

As such we will load in two variants of our data:
- one unlagged dataset
- one where all features except 'est' and 'year' are lagged by 1 year

In [ ]:
file_path = '../../src/data/temp/zbp_totals_with_features.csv'
data = pd.read_csv(file_path)

In [ ]:
file_path = '../../src/data/temp/lagged_zbp_totals_with_features.csv'
lagged_data = pd.read_csv(file_path)

# Drop Categorical Flags

In [ ]:
data = data.drop(columns=data.select_dtypes(exclude=['int64', 'float64']).columns)
lagged_data = lagged_data.drop(columns=lagged_data.select_dtypes(exclude=['int64', 'float64']).columns)

# Train-Test Split

Given our testing regimine, we will need two variants of each dataset
- Short-Term: Training [2012-2020], Testing [2021]
- Long-Term: Training [2012-2018], Testing [2019-2021]

In [ ]:
def train_test_split_by_year(data, end_year):
    data_train = data[data['year'] <= end_year]
    data_test = data[data['year'] > end_year]
    return data_train, data_test

### Short-Term

In [ ]:
end_year = 2020

short_data_train, short_data_test = train_test_split_by_year(data, end_year)
short_lagged_data_train, short_lagged_data_test = train_test_split_by_year(lagged_data, end_year)

### Long-Term

In [ ]:
end_year = 2018

long_data_train, long_data_test = train_test_split_by_year(data, end_year)
long_lagged_data_train, long_lagged_data_test = train_test_split_by_year(lagged_data, end_year)

# Standardization

In [ ]:
def standardize_data(data_train, data_test):
    train_mean = data_train.mean()
    train_mean['zip'] = 0
    train_std = data_train.std()
    train_std['zip'] = 1
    
    data_train_standardized = (data_train - train_mean) / train_std
    data_test_standardized = (data_test - train_mean) / train_std
    
    return data_train_standardized, data_test_standardized, (train_mean, train_std)

In [ ]:
def unstandardize_series(ser, mean, std):
    return (ser*std)+mean

In [ ]:
short_std_data_train, short_std_data_test, short_train_stats = standardize_data(short_data_train, short_data_test)
short_train_mean, short_train_std = short_train_stats

long_std_data_train, long_std_data_test, long_train_stats = standardize_data(long_data_train, long_data_test)
long_train_mean, long_train_std = long_train_stats

In [ ]:
short_lagged_std_data_train, short_lagged_std_data_test, short_lagged_train_stats = standardize_data(short_lagged_data_train, short_lagged_data_test)
short_lagged_train_mean, short_lagged_train_std = short_lagged_train_stats

long_lagged_std_data_train, long_lagged_std_data_test, long_lagged_train_stats = standardize_data(long_lagged_data_train, long_lagged_data_test)
long_lagged_train_mean, long_lagged_train_std = long_lagged_train_stats

# One Hot Encoding

In [ ]:
def convert_to_ohe(data_train, data_test):
    
    preproc = ColumnTransformer([('onehots', OneHotEncoder(handle_unknown='ignore'), ['zip'])]
                             ,remainder = 'passthrough')
    data_ohe_train = preproc.fit_transform(data_train)
    
    feature_names = preproc.get_feature_names_out()
    feature_names = np.char.replace(feature_names.astype('str'), 'onehots__','')
    feature_names = np.char.replace(feature_names, 'remainder__','')
    
    data_ohe_train = pd.DataFrame(data_ohe_train, columns=feature_names)
    
    data_ohe_test = preproc.transform(data_test)
    data_ohe_test = pd.DataFrame(data_ohe_test, columns=feature_names)
    
    return data_ohe_train, data_ohe_test

In [ ]:
short_ohe_data_train, short_ohe_data_test = convert_to_ohe(short_std_data_train, short_std_data_test)
long_ohe_data_train, long_ohe_data_test = convert_to_ohe(long_std_data_train, long_std_data_test)

short_lagged_ohe_data_train, short_lagged_ohe_data_test = convert_to_ohe(short_lagged_std_data_train, short_lagged_std_data_test)
long_lagged_ohe_data_train, long_lagged_ohe_data_test = convert_to_ohe(long_lagged_std_data_train, long_lagged_std_data_test)

### DL Test Set Creation

our tensorflow models require at least 1 previous timestamp to make predictions. To replicate testing procedure of our sklearn models add last timestep of training into test set.

In [ ]:
last_short_data_year = short_ohe_data_train['year'].unique().max()
tf_short_ohe_data_train = short_ohe_data_train[short_ohe_data_train['year']<last_short_data_year]
tf_short_ohe_data_test = pd.concat([short_ohe_data_train[short_ohe_data_train['year']==last_short_data_year], short_ohe_data_test])

In [ ]:
last_long_data_year = long_ohe_data_train['year'].unique().max()
tf_long_ohe_data_train = long_ohe_data_train[long_ohe_data_train['year']<last_long_data_year]
tf_long_ohe_data_test = pd.concat([long_ohe_data_train[long_ohe_data_train['year']==last_long_data_year], long_ohe_data_test])

# Feature Selection

### Corr Matrix

In [ ]:
top_k = 30
corr = short_lagged_ohe_data_train.corr()[['est']].sort_values(by='est', ascending=False)
vmin = corr.min()
vmax = corr.max()
corr_thresh = corr.abs().sort_values('est', ascending=False).iloc[top_k+2]['est']
corr = corr[corr['est'].abs() > corr_thresh]
# print(f'top {corr.shape[0]} features:')
corr_features = corr[1:-1]
display(corr[2:].style.background_gradient(cmap='coolwarm', vmin=vmin, vmax=vmax))
f'top {corr[2:].shape[0]} features by pearson correlation'

### Forward Feature Selection

In [ ]:
X_train = short_lagged_ohe_data_train.drop(columns=['est'])
y_train = short_lagged_ohe_data_train['est']
X_test = short_lagged_ohe_data_test.drop(columns=['est'])
y_test = short_lagged_ohe_data_test['est']

ffs = SequentialFeatureSelector(LinearRegression(n_jobs=-1), k_features=top_k, forward=True, n_jobs=-1)
ffs.fit(X_train, y_train)
ffs_features = list(ffs.k_feature_names_)
ffs_features = ffs_features[::-1]
ffs_features

# Models

In [ ]:
def fit_eval(model, data_train, data_test, included_feats, train_mean, train_std):
    
    if included_feats == 'all':
        included_feats = data_train.columns.drop(['est'])
    
    X_train = data_train[included_feats]
    y_train = data_train['est']
    X_test = data_test[included_feats]
    y_test = data_test['est']
    
    model.fit(X_train, y_train)
    
    y_preds = model.predict(X_train)
    inverted_y_train = unstandardize_series(y_train, train_mean['est'], train_std['est'])
    inverted_y_preds = unstandardize_series(y_preds, train_mean['est'], train_std['est'])
    train_rmse = mean_squared_error(inverted_y_train, inverted_y_preds, squared=False)
    
    y_preds = model.predict(X_test)
    inverted_y_test = unstandardize_series(y_test, train_mean['est'], train_std['est'])
    inverted_y_preds = unstandardize_series(y_preds, train_mean['est'], train_std['est'])
    test_rmse = mean_squared_error(inverted_y_test, inverted_y_preds, squared=False)
    
    return model, train_rmse, test_rmse

In [ ]:
all_features = 1

### Random Forest

In [ ]:
def run_grid_search(data_train, data_test, included_feats, model, param_grid):
    
    if included_feats == 'all':
        included_feats = data_train.columns.drop(['est'])

    X_train = data_train[included_feats]
    y_train = data_train['est']
    X_test = data_test[included_feats]
    y_test = data_test['est']
    
    grid_search = GridSearchCV(model, param_grid, scoring = 'neg_root_mean_squared_error', n_jobs = -1)
    grid_search.fit(X_train, y_train)
    
    return grid_search

In [ ]:
param_grid = {'n_estimators': [50], 
              'max_depth': [50]}

rf = RandomForestRegressor(random_state=SEED, n_jobs=-1)

gs_results = run_grid_search(short_lagged_ohe_data_train, short_lagged_ohe_data_test, 'all', rf, param_grid)
display(gs_results.best_params_)

short_rf = RandomForestRegressor(**gs_results.best_params_, random_state=SEED)
short_rf, short_rf_train_rmse, short_rf_test_rmse = fit_eval(short_rf, short_lagged_ohe_data_train, short_lagged_ohe_data_test, 
                                                             'all', 
                                                             short_lagged_train_mean, short_lagged_train_std)
print('train_rmse: ', short_rf_train_rmse)
print('test_rmse: ', short_rf_test_rmse)

with open('../../out/models/short_rf.pkl','wb') as f:
    pickle.dump(short_rf, f)

In [ ]:
rf = RandomForestRegressor(random_state=SEED, n_jobs=-1)

gs_results = run_grid_search(long_lagged_ohe_data_train, long_lagged_ohe_data_test, 'all', rf, param_grid)
display(gs_results.best_params_)

long_rf = RandomForestRegressor(**gs_results.best_params_, random_state=SEED)
long_rf, long_rf_train_rmse, long_rf_test_rmse = fit_eval(long_rf, long_lagged_ohe_data_train, long_lagged_ohe_data_test, 
                                                          'all', 
                                                          long_lagged_train_mean, long_lagged_train_std)
print('train_rmse: ', long_rf_train_rmse)
print('test_rmse: ', long_rf_test_rmse)

with open('../../out/models/long_rf.pkl','wb') as f:
    pickle.dump(short_rf, f)

In [ ]:
feature_names = short_lagged_ohe_data_train.columns.drop(['est'])

importances = long_rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in long_rf.estimators_], axis=0)

top_features = pd.Series(importances, index=feature_names).sort_values(ascending=False)[:top_k].sort_values(ascending=True)
forest_importances = top_features[:-1]
display(forest_importances.index.to_numpy())

fig, ax = plt.subplots()
forest_importances.plot(kind='barh', ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_xlabel("Mean decrease in impurity")

In [ ]:
mdi_top_features = top_features.index[::-1]
mdi_top_features

### Lin Reg

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        selected_features = [col for col in X.columns if any(name in col for name in self.feature_names)]
        return X[selected_features]

In [ ]:
all_feats = long_lagged_ohe_data_train.columns.drop(['est'])
preproc = ColumnTransformer([('feature_selector', FeatureSelector(feature_names=[]), all_feats)]
                             ,remainder = 'drop')
pl = Pipeline(steps=[('preproc', preproc), ('reg', LinearRegression(n_jobs=-1))])
param_grid = {'preproc__feature_selector__feature_names': [corr_features, ffs_features, mdi_top_features, all_feats]}

gs_results = run_grid_search(short_lagged_ohe_data_train, short_lagged_ohe_data_test, 'all', pl, param_grid)

short_lr = gs_results.best_estimator_
short_lr, short_lr_train_rmse, short_lr_test_rmse = fit_eval(short_lr, short_lagged_ohe_data_train, short_lagged_ohe_data_test, 
                                                             'all', 
                                                             short_lagged_train_mean, short_lagged_train_std)
print('train_rmse: ', short_lr_train_rmse)
print('test_rmse: ', short_lr_test_rmse)

with open('../../out/models/short_lr.pkl','wb') as f:
    pickle.dump(short_lr, f)

In [ ]:
preproc = ColumnTransformer([('feature_selector', FeatureSelector(feature_names=[]), all_feats)]
                             ,remainder = 'drop')
pl = Pipeline(steps=[('preproc', preproc), ('reg', LinearRegression(n_jobs=-1))])

gs_results = run_grid_search(short_lagged_ohe_data_train, short_lagged_ohe_data_test, 'all', pl, param_grid)

long_lr = gs_results.best_estimator_
long_lr, long_lr_train_rmse, long_lr_test_rmse = fit_eval(short_lr, long_lagged_ohe_data_train, long_lagged_ohe_data_test, 
                                                          'all', 
                                                          long_lagged_train_mean, long_lagged_train_std)
print('train_rmse: ', long_lr_train_rmse)
print('test_rmse: ', long_lr_test_rmse)

with open('../../out/models/long_lr.pkl','wb') as f:
    pickle.dump(long_lr, f)

### Lasso

In [ ]:
short_lasso = LassoCV(random_state=SEED)
short_lasso, short_lasso_train_rmse, short_lasso_test_rmse = fit_eval(short_lasso, short_lagged_ohe_data_train, short_lagged_ohe_data_test, 
                                                                      'all', 
                                                                      short_lagged_train_mean, short_lagged_train_std)
print('train_rmse: ', short_lasso_train_rmse)
print('test_rmse: ', short_lasso_test_rmse)

with open('../../out/models/short_lasso.pkl','wb') as f:
    pickle.dump(short_lasso, f)

In [ ]:
long_lasso = LassoCV(random_state=SEED)
long_lasso, long_lasso_train_rmse, long_lasso_test_rmse = fit_eval(long_lasso, long_lagged_ohe_data_train, long_lagged_ohe_data_test, 
                                                                   'all', 
                                                                   long_lagged_train_mean, long_lagged_train_std)
print('train_rmse: ', long_lasso_train_rmse)
print('test_rmse: ', long_lasso_test_rmse)

with open('../../out/models/long_lasso.pkl','wb') as f:
    pickle.dump(long_lasso, f)

### ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

class ARIMAForecast():
    
    def __init__(self, data, n_lag_terms ,diff_order ,window_size):
        self.data = data
        self.models = {}
        self.n_lag_terms = n_lag_terms
        self.diff_order = diff_order
        self.window_size = window_size
        
    def train(self):
        for zip_code in self.data['zip'].unique():
            # filter
            curr_data = self.data[self.data['zip']==zip_code][['year', 'est']].set_index('year')
            start_time = curr_data.index[0]
            # train
            model = ARIMA(curr_data, order=(self.n_lag_terms ,self.diff_order ,self.window_size))
            try:
                results = model.fit()
                self.models[zip_code] = (results, start_time)
            except:
                pass
            
    def forecast(self, year):
        preds = []
        # last year seen in the training set
        # used to calculate start range for forecast, to avoid predicting values in training set
        data_last_year = self.data['year'].max()
        for zip_code, model_info in self.models.items():
            model, start_time = model_info
            # make predictions
            curr_pred = model.predict(data_last_year-start_time+1,year-start_time)
            # modify results into a df object
            curr_pred = curr_pred.to_frame().assign(zip=np.full(curr_pred.shape[0], zip_code)).reset_index()
            curr_pred = curr_pred.rename(columns={'index':'year', 0:'est', 'predicted_mean':'est'})
            # address issue where timestamp of some predictions is the number of years after the last year
            # in the training data rather than a timestamp object
            max_int = curr_pred[curr_pred['year'].apply(lambda x: type(x) == int)]['year'].max()
            curr_pred['year'] = curr_pred['year'].apply(lambda x: year-max_int+x if (type(x) == int) else x)
            preds += [curr_pred]
            
        return pd.concat(preds, ignore_index=True).reset_index(drop=True)
            

In [ ]:
model = ARIMAForecast(short_data_train, 1, 1, 1)

model.train()

forecast = model.forecast(short_data_test['year'].max())
preds_labels = forecast.merge(short_data_test, on=['zip', 'year'], suffixes=('_pred', '_true'))

short_arima_train_rmse = None
short_arima_test_rmse = mean_squared_error(preds_labels['est_true'], preds_labels['est_pred'], squared=False)

with open('../../out/models/short_arima.pkl','wb') as f:
    pickle.dump(model, f)

In [ ]:
model = ARIMAForecast(long_data_train, 1, 1, 1)

model.train()

forecast = model.forecast(long_data_test['year'].max())
preds_labels = forecast.merge(long_data_test, on=['zip', 'year'], suffixes=('_pred', '_true'))

long_arima_train_rmse = None
long_arima_test_rmse = mean_squared_error(preds_labels['est_true'], preds_labels['est_pred'], squared=False)

with open('../../out/models/long_arima.pkl','wb') as f:
    pickle.dump(model, f)

# LSTM

#### Windowing

In [ ]:
class WindowGenerator():
    
    def __init__(self, input_width, label_width, shift,
                train_df=long_ohe_data_train, test_df=long_ohe_data_test,
                label_columns=None, batch_size=1):
        
        self.batch_size = batch_size
        
        # Store the raw data.
        self.train_df = train_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                          enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
    
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]

        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=False,
            batch_size=self.batch_size,)

        ds = ds.map(self.split_window)

        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result
    
    def plot(self, model=None, plot_col='est', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='Predictions',
                            c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('year')

In [ ]:
IN_STEPS = 1
OUT_STEPS = 1

single_step_window = WindowGenerator(input_width=IN_STEPS,
                                    label_width=OUT_STEPS,
                                    shift=OUT_STEPS,
                                    label_columns=['est'],
                                    batch_size=1)
single_step_window

In [ ]:
wide_window = WindowGenerator(input_width=5,
                              label_width=5,
                              shift=1,
                              label_columns=['est'],
                              batch_size=1)
wide_window

#### Splitting Data into ZIP Codes

In [ ]:
def split_by_zip_code(data_train, data_test, window, ignore_test=False):
    
    data_train_by_zc_tf = {}
    for zip_code in data_train.filter(like='zip').columns:
        data_by_zc = data_train[data_train[zip_code]==1]
        data_train_by_zc_tf[zip_code] = window.make_dataset(data_by_zc)
        
    
    data_test_by_zc_tf = {}
    
    if not ignore_test:
        for zip_code in data_test.filter(like='zip').columns:
            data_by_zc = data_test[data_test[zip_code]==1]
            data_test_by_zc_tf[zip_code] = window.make_dataset(data_by_zc)
        
    return data_train_by_zc_tf, data_test_by_zc_tf

In [ ]:
short_data_train_by_zc_tf, short_data_test_by_zc_tf = split_by_zip_code(short_ohe_data_train, short_ohe_data_test, single_step_window, ignore_test=True)
long_data_train_by_zc_tf, long_data_test_by_zc_tf = split_by_zip_code(long_ohe_data_train, long_ohe_data_test, single_step_window)

In [ ]:
wide_short_data_train_by_zc_tf, wide_short_data_test_by_zc_tf = split_by_zip_code(short_ohe_data_train, short_ohe_data_test, wide_window, ignore_test=True)
wide_long_data_train_by_zc_tf, wide_long_data_test_by_zc_tf = split_by_zip_code(long_ohe_data_train, long_ohe_data_test, wide_window)

# TF MODELS

In [ ]:
def evaluate_on_all_zip(model, data_train_by_zc):
    total = 0
    i = 0
    for zc in data_train_by_zc.keys():
        total += unstandardize_series(model.evaluate(data_train_by_zc[zc], verbose=0)[0], 
                                      long_train_mean['est'], long_train_std['est'])
        i += 1
    return np.sqrt(total/i)

In [ ]:
def wide_plot_model(model, wide_data, window):
    inputs, labels = next(iter(wide_data['zip_91915.0']))

    plt.figure(figsize=(12, 8))
    plot_col_index = window.column_indices['est']

    plt.ylabel(f'est [normed]')
    plt.plot(window.input_indices, inputs[0, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if window.label_columns:
        label_col_index = window.label_columns_indices.get('est', None)
    else:
        label_col_index = plot_col_index

    plt.scatter(window.label_indices, labels[0, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)

    predictions = model(inputs)
    print(predictions[0, :, label_col_index])
    plt.scatter(window.label_indices, predictions[0, :, label_col_index],
                marker='X', edgecolors='k', label='Predictions',
                c='#ff7f0e', s=64)

In [ ]:
def compile_and_fit(model, data_train_by_zc, data_test_by_zc, num_epochs):
    
    KERAS_VERBOSITY = 0
    patience = 4

    losses = []
    val_losses = []

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.losses.MeanSquaredError()])
    
    for epoch in tqdm(np.arange(num_epochs)):
        
        if (len(losses) >= 2) and (np.abs(losses[-1] - losses[-2]) < 0.1):
            patience -= 1
        if patience <= 0:
            break
        
        loss_curr_epoch = 0
        val_loss_curr_epoch = 0
        i = 0
        
        data_train_by_zip = list(data_train_by_zc.values())
        data_test_by_zip = list(data_test_by_zc.values())
        
        for i in np.arange(len(data_train_by_zip)):
            
            history = model.fit(data_train_by_zip[i], epochs=1, validation_data=data_train_by_zip[i], verbose=KERAS_VERBOSITY)
            loss_curr_epoch += history.history['loss'][0]
            val_loss_curr_epoch += history.history['val_loss'][0]
            i += 1
                
        losses += [np.sqrt(unstandardize_series(loss_curr_epoch/i, long_train_mean['est'], long_train_std['est']))]
        val_losses += [np.sqrt(unstandardize_series(val_loss_curr_epoch/i, long_train_mean['est'], long_train_std['est']))]
                
    return losses, val_losses

In [ ]:
MAX_EPOCHS = 2

### Baseline

In [ ]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [ ]:
column_indices = {name: i for i, name in enumerate(long_ohe_data_train.columns)}
baseline = Baseline(label_index=column_indices['est'])
baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.losses.MeanSquaredError()])

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', baseline(wide_window.example[0]).shape)

In [ ]:
wide_plot_model(baseline, wide_long_data_train_by_zc_tf, wide_window)

#### Linear

In [ ]:
short_linear = tf.keras.Sequential([tf.keras.layers.Dense(units=1)])
long_linear = tf.keras.Sequential([tf.keras.layers.Dense(units=1)])

In [ ]:
print('Input shape:', single_step_window.example[0].shape)
print('Output shape:', long_linear(single_step_window.example[0]).shape)

In [ ]:
losses, val_losses = compile_and_fit(short_linear, short_data_train_by_zc_tf, short_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
losses, val_losses = compile_and_fit(long_linear, long_data_train_by_zc_tf, long_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', long_linear(wide_window.example[0]).shape)

In [ ]:
wide_plot_model(long_linear, wide_long_data_train_by_zc_tf, wide_window)

### Dense

In [ ]:
short_dense = tf.keras.Sequential([tf.keras.layers.Dense(units=256, activation='relu'),
                                   tf.keras.layers.Dense(units=128, activation='relu'),
                                   tf.keras.layers.Dense(units=1)])
long_dense = tf.keras.Sequential([tf.keras.layers.Dense(units=256, activation='relu'),
                                   tf.keras.layers.Dense(units=128, activation='relu'),
                                   tf.keras.layers.Dense(units=1)])

In [ ]:
print('Input shape:', single_step_window.example[0].shape)
print('Output shape:', long_dense(single_step_window.example[0]).shape)

In [ ]:
losses, val_losses = compile_and_fit(short_dense, short_data_train_by_zc_tf, short_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
losses, val_losses = compile_and_fit(long_dense, long_data_train_by_zc_tf, long_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', long_dense(wide_window.example[0]).shape)

In [ ]:
wide_plot_model(long_dense, wide_long_data_train_by_zc_tf, wide_window)

### RNN

In [ ]:
short_lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(256, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])
long_lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', long_lstm_model(wide_window.example[0]).shape)

In [ ]:
losses, val_losses = compile_and_fit(short_lstm_model, wide_short_data_train_by_zc_tf, wide_short_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
losses, val_losses = compile_and_fit(long_lstm_model, wide_long_data_train_by_zc_tf, wide_long_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', long_lstm_model(wide_window.example[0]).shape)

In [ ]:
wide_plot_model(long_lstm_model, wide_long_data_train_by_zc_tf, wide_window)

# MultiStep Models

In [ ]:
OUT_STEPS = 3
multi_window = WindowGenerator(input_width=OUT_STEPS,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multi_window.plot()
multi_wide_short_data_train_by_zc_tf, multi_wide_short_data_test_by_zc_tf = split_by_zip_code(short_ohe_data_train, short_ohe_data_test, multi_window, ignore_test=True)
multi_wide_long_data_train_by_zc_tf, multi_wide_long_data_test_by_zc_tf = split_by_zip_code(long_ohe_data_train, long_ohe_data_test, multi_window)
multi_window

### Autoregressive LSTM

In [ ]:
num_features = long_ohe_data_train.shape[1]

In [ ]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
def warmup(self, inputs):
    # inputs.shape => (batch, time, features)
    # x.shape => (batch, lstm_units)
    x, *state = self.lstm_rnn(inputs)

    # predictions.shape => (batch, features)
    prediction = self.dense(x)
    return prediction, state

FeedBack.warmup = warmup

In [ ]:
def call(self, inputs, training=None):
    # Use a TensorArray to capture dynamically unrolled outputs.
    predictions = []
    # Initialize the LSTM state.
    prediction, state = self.warmup(inputs)

    # Insert the first prediction.
    predictions.append(prediction)

    # Run the rest of the prediction steps.
    for n in range(1, self.out_steps):
        # Use the last prediction as input.
        x = prediction
        # Execute one lstm step.
        x, state = self.lstm_cell(x, states=state,
                                  training=training)
        # Convert the lstm output to a prediction.
        prediction = self.dense(x)
        # Add the prediction to the output.
        predictions.append(prediction)

    # predictions.shape => (time, batch, features)
    predictions = tf.stack(predictions)
    # predictions.shape => (batch, time, features)
    predictions = tf.transpose(predictions, [1, 0, 2])
    return predictions

FeedBack.call = call

In [ ]:
short_feedback_model = FeedBack(units=256, out_steps=OUT_STEPS)
long_feedback_model = FeedBack(units=256, out_steps=OUT_STEPS)

In [ ]:
losses, val_losses = compile_and_fit(short_feedback_model, multi_wide_short_data_train_by_zc_tf, multi_wide_short_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
losses, val_losses = compile_and_fit(long_feedback_model, multi_wide_long_data_train_by_zc_tf, multi_wide_long_data_test_by_zc_tf, MAX_EPOCHS)
plt.plot(np.arange(1, len(losses) + 1), losses, label='train')
plt.plot(np.arange(1, len(val_losses) + 1), val_losses, label='validation')

In [ ]:
print('Input shape:', multi_window.example[0].shape)
print('Output shape:', long_feedback_model(multi_window.example[0]).shape)

In [ ]:
def auto_wide_plot_model(model, wide_data, window, extra_steps):
    inputs, labels = next(iter(wide_data['zip_91915.0']))

    plt.figure(figsize=(12, 8))
    plot_col_index = window.column_indices['est']

    plt.ylabel(f'est [normed]')
    plt.plot(window.input_indices, inputs[0, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if window.label_columns:
        label_col_index = window.label_columns_indices.get('est', None)
    else:
        label_col_index = plot_col_index

    plt.scatter(window.label_indices, labels[0, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)

    model.out_steps += extra_steps
    predictions = model(inputs, training=False)
    model.out_steps -= extra_steps
    new_pred_indicies = np.append(multi_window.label_indices, np.arange(multi_window.label_indices[-1] + 1, multi_window.label_indices[-1] + 1 + extra_steps))
    plt.scatter(new_pred_indicies, predictions[0, :, label_col_index],
                marker='X', edgecolors='k', label='Predictions',
                c='#ff7f0e', s=64)

In [ ]:
auto_wide_plot_model(long_feedback_model, multi_wide_long_data_train_by_zc_tf, multi_window, 10)

In [ ]:
def sum_auto_wide_plot_model(model, wide_data, window, extra_steps):
    
    total_preds = None
    total_inputs = None
    plot_col_index = window.column_indices['est']
    if window.label_columns:
        label_col_index = window.label_columns_indices.get('est', None)
    else:
        label_col_index = plot_col_index
    model.out_steps += extra_steps
    
    for zc in wide_data.keys():
        
        inputs, labels = next(iter(wide_data[zc]))
        
        predictions = model(inputs, training=False)
        curr_preds = unstandardize_series(predictions[0, :, label_col_index], long_train_mean['est'], long_train_std['est'])
        curr_inputs = unstandardize_series(inputs[0, :, plot_col_index], long_train_mean['est'], long_train_std['est'])
        
        if total_preds is None:
            total_preds = curr_preds
        else:
            total_preds += curr_preds
            
        if total_inputs is None:
            total_inputs = curr_inputs
        else:
            total_inputs += curr_inputs
            
    model.out_steps -= extra_steps
    
    return total_inputs, total_preds

In [ ]:
auto_regressive_steps = 2200-2017
sum_inputs, sum_preds = sum_auto_wide_plot_model(long_feedback_model, multi_wide_long_data_train_by_zc_tf, multi_window, auto_regressive_steps)

input_indicies = np.arange(2012, 2012 + sum_inputs.shape[0])
preds_indicies = np.arange(input_indicies[0] + 3, input_indicies[-1] + auto_regressive_steps + 4)

plt.plot(input_indicies, sum_inputs, marker='o')
plt.plot(preds_indicies, sum_preds, marker='X', color='#ff7f0e')

# TF MODEL EVALUATION

In [ ]:
test_uni_window = WindowGenerator(input_width=1,
                                  label_width=1,
                                  shift=1,
                                  label_columns=['est'],
                                  batch_size=1)
test_uni_window

In [ ]:
test_uni_short_data_train_by_zc_tf, test_uni_short_data_test_by_zc_tf = split_by_zip_code(tf_short_ohe_data_train, tf_short_ohe_data_test, test_uni_window)
test_uni_long_data_train_by_zc_tf, test_uni_long_data_test_by_zc_tf = split_by_zip_code(tf_long_ohe_data_train, tf_long_ohe_data_test, test_uni_window)

In [ ]:
models_to_test = [('baseline', baseline, baseline), ('linear', short_linear, long_linear), 
                  ('dense', short_dense, long_dense), ('lstm', short_lstm_model, long_lstm_model), 
                  ('autoregressive-lstm', short_feedback_model, long_feedback_model)]

testing_scenarios = [('short-term', test_uni_short_data_train_by_zc_tf, test_uni_short_data_test_by_zc_tf),
                     ('long-term', test_uni_long_data_train_by_zc_tf, test_uni_long_data_test_by_zc_tf)]

In [ ]:
eval_info = []
for model_name, short_model, long_model in models_to_test:
    model_eval = [model_name]
    for scenario_name, train_data, test_data in testing_scenarios:
        if scenario_name == 'short-term':
            model = short_model
        else:
            model = long_model
        train_rmse = evaluate_on_all_zip(model, train_data)
        test_rmse = evaluate_on_all_zip(model, test_data)
        model_eval += [train_rmse]
        model_eval += [test_rmse]
            
    eval_info += [model_eval]

In [ ]:
eval_info += [['lr', short_lr_train_rmse, short_lr_test_rmse,
                     long_lr_train_rmse, long_lr_test_rmse]]
eval_info += [['rf', short_rf_train_rmse, short_rf_test_rmse,
                     long_rf_train_rmse, long_rf_test_rmse]]
eval_info += [['lasso', short_lasso_train_rmse, short_lasso_test_rmse,
                        long_lasso_train_rmse, long_lasso_test_rmse]]
eval_info += [['arima', short_arima_train_rmse, short_arima_test_rmse,
                        long_arima_train_rmse, long_arima_test_rmse]]

In [ ]:
evals_df = pd.DataFrame(eval_info, columns=['model', 
                                           'short-term train rmse', 'short-term test rmse',
                                           'long-term train rmse', 'long-term test rmse'])
evals_df

In [ ]:
evals_df[['model', 'short-term test rmse', 'long-term test rmse']]

# Save Model

In [ ]:
short_model_filepath = '../../out/models/short_feedback_model_weights.tf'
long_model_filepath = '../../out/models/long_feedback_model_weights.tf'

In [ ]:
# short_feedback_model.save_weights(short_model_filepath)
# long_feedback_model.save_weights(model_filepath)

In [ ]:
loaded_short_feedback = FeedBack(units=256, out_steps=OUT_STEPS)
loaded_short_feedback.built = True
loaded_short_feedback.load_weights(short_model_filepath)

loaded_long_feedback = FeedBack(units=256, out_steps=OUT_STEPS)
loaded_long_feedback.built = True
loaded_long_feedback.load_weights(long_model_filepath)

In [ ]:
auto_regressive_steps = 2200-2017
sum_inputs, sum_preds = sum_auto_wide_plot_model(loaded_short_feedback, multi_wide_long_data_train_by_zc_tf, multi_window, auto_regressive_steps)

input_indicies = np.arange(2012, 2012 + sum_inputs.shape[0])
preds_indicies = np.arange(input_indicies[0] + 3, input_indicies[-1] + auto_regressive_steps + 4)

plt.plot(input_indicies, sum_inputs, marker='o')
plt.plot(preds_indicies, sum_preds, marker='X', color='#ff7f0e')

In [ ]:
auto_regressive_steps = 2200-2017
sum_inputs, sum_preds = sum_auto_wide_plot_model(loaded_long_feedback, multi_wide_long_data_train_by_zc_tf, multi_window, auto_regressive_steps)

input_indicies = np.arange(2012, 2012 + sum_inputs.shape[0])
preds_indicies = np.arange(input_indicies[0] + 3, input_indicies[-1] + auto_regressive_steps + 4)

plt.plot(input_indicies, sum_inputs, marker='o')
plt.plot(preds_indicies, sum_preds, marker='X', color='#ff7f0e')